In [1]:
# Import class
import os
import pickle
import sys
parent_dir = os.path.dirname(os.path.dirname(os.getcwd()))
class_dir = os.path.join(parent_dir, 'Scripts')
sys.path.insert(0, class_dir)
import NBAStats
stats_file = os.path.join(parent_dir, 'Data', 'nba_stats.pickle')
print(stats_file)
nba_stats = pickle.load(open(stats_file, 'rb'))

import player_market_class
import scraper_SB
import empirical
import pandas as pd

C:\Users\johnn\Desktop\github\nba_stats\Data\nba_stats.pickle


In [2]:
links = scraper_SB.get_links_SB('08/01/19')
for link in links:
    print(link)

https://www.sportsbet.com.au/betting/basketball-us/nba-matches/memphis-grizzlies-at-new-orleans-pelicans-4451968
https://www.sportsbet.com.au/betting/basketball-us/nba-matches/utah-jazz-at-milwaukee-bucks-4451974
https://www.sportsbet.com.au/betting/basketball-us/nba-matches/denver-nuggets-at-houston-rockets-4451975
https://www.sportsbet.com.au/betting/basketball-us/nba-matches/los-angeles-lakers-at-dallas-mavericks-4451976
https://www.sportsbet.com.au/betting/basketball-us/nba-matches/orlando-magic-at-sacramento-kings-4451977
https://www.sportsbet.com.au/betting/basketball-us/nba-matches/new-york-knicks-at-portland-trail-blazers-4451978


In [3]:
past_n_games = 30

In [4]:
for link in links:
    print(link)
    player_markets = scraper_SB.get_player_markets_SB(link)

    edge_list = []
    player_list = []
    over_under_list = []
    baseline_list = []
    stat_list = []
    odds_list = []
    theo_list = []

    for player_market in player_markets:
        # Convert Sportsbet odds into class
        try:
            # If unable to parse info to class, move onto next one
            p_mrkt = player_market_class.PlayerMarketSportsBet(player_market)
        except:
            print('Unable to parse:\n {}'.format(player_market))
        
        try:
            # If unable to calculate theo (e.g. if player's name not correct), move onto next one
            # Calculate theo odds
            theo_odds_over, theo_odds_under = empirical.empirical_odds(p_mrkt.get_player(), p_mrkt.get_stat(), 
                                     p_mrkt.get_baseline(), past_n_games)
        except:
            print('Unable to calculate theo for {} {}'.format(p_mrkt.get_player(),
                                                          p_mrkt.get_stat()))
            continue
            
        theo_pr_over = 1/theo_odds_over
        theo_pr_under = 1 - theo_pr_over

        # Calculate edge
        edge_over = theo_pr_over * p_mrkt.get_odds_over() - 1
        edge_under = theo_pr_under * p_mrkt.get_odds_under() - 1

        if edge_over > edge_under:
            best_edge = edge_over
            over_under = 'over'
            theo_list.append(theo_odds_over)
            odds_list.append(p_mrkt.get_odds_over())
        else:
            best_edge = edge_under
            over_under = 'under'
            theo_list.append(theo_odds_under)
            odds_list.append(p_mrkt.get_odds_under())
        
        edge_list.append(best_edge)
        player_list.append(p_mrkt.get_player())
        over_under_list.append(over_under)
        baseline_list.append(p_mrkt.get_baseline())
        stat_list.append(p_mrkt.get_stat())
    
    # Create df, sort by edge and print
    df_game_mrkts_edge = pd.DataFrame({
        'Edge':edge_list,
        'Player':player_list,
        'over_under':over_under_list,
        'Baseline':baseline_list,
        'Stat':stat_list,
        'Odds':odds_list,
        'Theo':theo_list
    }, columns = ['Edge', 'Player', 'over_under', 'Baseline', 'Stat', 'Odds', 'Theo'])
    
    df_game_mrkts_edge = df_game_mrkts_edge.sort_values(by='Edge', ascending=False)
    print(df_game_mrkts_edge)
    print('')

https://www.sportsbet.com.au/betting/basketball-us/nba-matches/memphis-grizzlies-at-new-orleans-pelicans-4451968
Unable to calculate theo for MIKE gONLEY PTS
Unable to calculate theo for MIKE gONLEY REB
Unable to calculate theo for MIKE gONLEY AST
        Edge             Player over_under  Baseline Stat  Odds      Theo
11  0.358333       Jrue Holiday       over       3.5  REB  1.63  1.200000
15  0.320000       Jrue Holiday       over       6.5  AST  1.80  1.363636
13  0.224000      Anthony Davis       over       3.5  AST  2.04  1.666667
16  0.188000      Elfrid Payton       over       5.5  AST  1.98  1.666667
5   0.171667       Jrue Holiday       over      19.5  PTS  1.85  1.578947
12  0.120000         Marc Gasol      under       5.5  AST  1.60  1.428571
4   0.102000      Julius Randle       over      20.5  PTS  1.74  1.578947
8   0.072000  Jaren Jackson Jr.      under       5.5  REB  2.01  1.875000
0   0.056000         Marc Gasol      under      15.5  PTS  1.98  1.875000
1   0.037000

KeyboardInterrupt: 